# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Scrape and clean the data from wikipedia into a pandas DataFrame

In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [4]:
wiki_table = soup.find('table')
wiki_headers= wiki_table.findAll('th')
wiki_data= wiki_table.findAll('td')

In [5]:
clean_data = [data.text.strip() for data in wiki_data]
clean_headers = [data.text.strip() for data in wiki_headers]

In [6]:
clean_data[0],clean_data[1],clean_data[2]

('M1A', 'Not assigned', '')

In [7]:
Postal_code_list = []
Borough_list = []
Neighborhood_list = []

for i, data in enumerate(clean_data):
    if i % 3 == 0:
        Postal_code_list.append(data.replace(" /",","))
    elif i % 3 == 1:
        Borough_list.append(data.replace(" /",","))
    elif i % 3 == 2:
        Neighborhood_list.append(data.replace(" /",","))       

In [8]:
len(Postal_code_list),len(Borough_list),len(Neighborhood_list)

(180, 180, 180)

In [9]:
df = pd.DataFrame()
df[clean_headers[0]]=Postal_code_list
df[clean_headers[1]]=Borough_list
df[clean_headers[2]]=Neighborhood_list
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df.shape

(180, 3)

In [11]:
df = df[~df.Borough.str.contains("Not assigned")]
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
df.shape

(103, 3)

In [13]:
df.reset_index(drop=True,inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Part 2- Add columns for Latitude and Longitude for each postal code

In [14]:
!pip install geocoder
import geocoder # import geocoder

In [15]:
latitude_list=[]
longitude_list=[]

for code in df['Postal code']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
    latitude_list.append(lat_lng_coords[0])
    longitude_list.append(lat_lng_coords[1])

In [16]:
len(latitude_list),len(longitude_list)

(103, 103)

In [17]:
df['Latitude']=latitude_list
df['Longitude']=longitude_list
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [18]:
df.shape

(103, 5)

In [19]:
#Test to verify using the postal code= 'M3A'
lat_lng_coords = None
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format('M3A'))
    lat_lng_coords = g.latlng
(lat_lng_coords[0],lat_lng_coords[1])

(43.75293455500008, -79.33564142299997)

## Part 3- Create a Folium map with pinned Neighborhoods, extract venue data from FourSquare and use KNN to cluster venue data

In [20]:
!pip install folium
import folium
from pandas.io.json import json_normalize

In [21]:
tor_latitude = df.Latitude.mean()
tor_longitude = df.Longitude.mean()
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
df_toronto_only = df[df.Borough.str.contains("Toronto")]
df_toronto_only.reset_index(drop=True,inplace=True)
df_toronto_only.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


In [23]:
df_toronto_only.shape

(39, 5)

In [24]:
tor_latitude = df_toronto_only.Latitude.mean()
tor_longitude = df_toronto_only.Longitude.mean()
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto_only['Latitude'], df_toronto_only['Longitude'], df_toronto_only['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
# The code was removed by Watson Studio for sharing.

In [26]:
neighborhood_latitude = df_toronto_only.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_only.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto_only.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65096410900003, -79.35304116399999.


In [27]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [28]:
results = requests.get(url).json()
#results

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
1,Young Centre for the Performing Arts,Performing Arts Venue,43.650825,-79.357593
2,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
3,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843
4,BATLgrounds,Athletics & Sports,43.647088,-79.351306


In [31]:
nearby_venues.shape

(24, 4)

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=df_toronto_only['Neighborhood'], latitudes=df_toronto_only['Latitude'], longitudes=df_toronto_only['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
 Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(1587, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports


In [35]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,66,66,66,66,66,66
"Brockton, Parkdale Village, Exhibition Place",42,42,42,42,42,42
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst\n Quay, South Niagara, Island airport",65,65,65,65,65,65
Central Bay Street,52,52,52,52,52,52
Christie,12,12,12,12,12,12
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_onehot.shape

(1587, 221)

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.015152,0.00000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,...,0.000000,0.00000,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.02381,0.000000,0.023810,0.023810,0.000000,0.000000,0.000000,0.000000,...,0.047619,0.02381,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.00000,0.030000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00000,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.019231,0.019231,0.019231,0.000000
5,Christie,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012821,0.00000,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.00000,0.040000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.037037,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
toronto_grouped.shape

(38, 221)

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.05
3      Farmers Market  0.03
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.10
1                    Café  0.07
2               Gift Shop  0.05
3  Thrift / Vintage Store  0.05
4     Japanese Restaurant  0.02


----Business reply mail Processing CentrE----
                 venue  freq
0          Coffee Shop  0.08
1                Hotel  0.05
2           Restaurant  0.04
3  Japanese Restaurant  0.04
4                 Café  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
 Quay, South Niagara, Island airport----
               venue  freq
0        Coffee Shop  0.08
1               Café  0.06
2         Restaurant  0.06
3               Park  0.05
4  French Restaurant  0.05


----Central Bay Street----
             venue  freq
0      Co

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Lounge,Cheese Shop,Restaurant,Italian Restaurant,Bakery,Beer Bar,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Gift Shop,Thrift / Vintage Store,French Restaurant,Indian Restaurant,Brewery,Italian Restaurant,Japanese Restaurant,Sandwich Place
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Café,American Restaurant,Seafood Restaurant,Movie Theater,Gym,Steakhouse
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Restaurant,French Restaurant,Park,Lounge,Speakeasy,Bar,Italian Restaurant,Gym / Fitness Center
4,Central Bay Street,Coffee Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Clothing Store,Plaza,Breakfast Spot,Japanese Restaurant,Sandwich Place,Shopping Mall


In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_only

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0.0,Pub,Café,Athletics & Sports,Seafood Restaurant,Thai Restaurant,Chocolate Shop,Coffee Shop,French Restaurant,Distribution Center,Bank
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,0.0,Coffee Shop,Sushi Restaurant,Café,Hobby Shop,Discount Store,Pizza Place,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Bar,Sandwich Place,Hotel,Cosmetics Shop,Tea Room,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0.0,Coffee Shop,Café,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Cocktail Bar,Italian Restaurant,Department Store,Creperie
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0.0,Coffee Shop,Trail,Pub,Asian Restaurant,Church,Health Food Store,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Donut Shop


In [46]:
toronto_merged.dropna(inplace=True)

In [48]:
# create map
tor_latitude = df_toronto_only.Latitude.mean()
tor_longitude = df_toronto_only.Longitude.mean()
map_clusters = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Pub,Café,Athletics & Sports,Seafood Restaurant,Thai Restaurant,Chocolate Shop,Coffee Shop,French Restaurant,Distribution Center,Bank
1,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Café,Hobby Shop,Discount Store,Pizza Place,Pharmacy,Park,Middle Eastern Restaurant,Juice Bar
2,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Bar,Sandwich Place,Hotel,Cosmetics Shop,Tea Room,Fast Food Restaurant
3,Downtown Toronto,0.0,Coffee Shop,Café,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Cocktail Bar,Italian Restaurant,Department Store,Creperie
4,East Toronto,0.0,Coffee Shop,Trail,Pub,Asian Restaurant,Church,Health Food Store,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Donut Shop
5,Downtown Toronto,0.0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Lounge,Cheese Shop,Restaurant,Italian Restaurant,Bakery,Beer Bar,Hotel
6,Downtown Toronto,0.0,Coffee Shop,Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Clothing Store,Plaza,Breakfast Spot,Japanese Restaurant,Sandwich Place,Shopping Mall
8,Downtown Toronto,0.0,Coffee Shop,Café,Restaurant,Clothing Store,Salad Place,American Restaurant,Gym,Sushi Restaurant,Deli / Bodega,Hotel
9,West Toronto,0.0,Park,Grocery Store,Café,Bus Line,Furniture / Home Store,Brazilian Restaurant,Bank,Bakery,Smoke Shop,Pharmacy
11,West Toronto,0.0,Restaurant,Cocktail Bar,Bar,Vietnamese Restaurant,Wine Bar,Asian Restaurant,Coffee Shop,Yoga Studio,Cupcake Shop,Seafood Restaurant


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1.0,Home Service,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2.0,Bus Line,Swim School,Women's Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,3.0,Grocery Store,Café,Park,Coffee Shop,Playground,Athletics & Sports,Candy Store,Baby Store,Farmers Market,Ethiopian Restaurant
12,East Toronto,3.0,Bus Line,Business Service,Park,Grocery Store,Discount Store,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
23,Central Toronto,3.0,Playground,Gym Pool,Park,Garden,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
33,Downtown Toronto,3.0,Playground,Park,Grocery Store,Candy Store,Women's Store,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4.0,Harbor / Marina,Theme Park,Park,Farm,Women's Store,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
